In [33]:
import re, subprocess, pyodbc , os
import numpy as np # linear algebra 
from sqlalchemy import func, create_engine
from config import mongoConn, connstr  
import pandas as pd 

In [38]:
engine = create_engine(connstr)
conn = engine.connect()
query = """ \
SELECT QPK, QGROUP, QuestionTypeCode, QTTable, ROW_NUMBER() OVER(ORDER BY IDTEXT ASC) -1 AS Row 
FROM  vw_MetricsCompositeKeys
WHERE PK_Form LIKE '%EINS%' 
ORDER BY IDTEXT ASC 
"""  
df = pd.read_sql(query,con=conn)  
conn.close()
df 

,QPK,QGROUP,QuestionTypeCode,QTTable,Row
0,20850,2110,NCEinsteinPublicIP,EinsteinPublicIP,0
1,20861,2111,NCEinsteinUnannounced,EinsteinUnannounced,1
2,20881,2113,NCEinsteinTaps,EinsteinTaps,2
3,20891,2114,NCEinsteinVLAN,EinsteinVLAN,3
4,20900,2115,NCEinsteinUnmonitoredTraffic,EinsteinUnmonitoredTraffic,4
5,20940,2120,NCEinsteinNATInfo,EinsteinNATInfo,5
6,20950,2121,NCEinsteinBGP,EinsteinBGP,6
7,20960,2122,NCEinsteinArtifactGrid,None,7


In [53]:
l = [] 
for i in range(2,9):  
    dff = pd.read_csv(fr'c:\temp\S{i}.csv')
    dff['SECTION'] = i 
    l.append(dff)
dfs=pd.concat(l)
dfs.columns = [c.replace(' ','') for c in dfs.columns]
dfs.columns 
#dfs

Index(['ColumnHeadings', 'DataEntryValidations', 'EnumeratedList',
       'AdditionalHelpText', 'SECTION', 'Feature'],
      dtype='object')

,ColumnHeadings,DataEntryValidations,EnumeratedList,AdditionalHelpText,SECTION,Feature
0,Starting IP,Mandatory Entry\n\nIPv4_Address Validation\nxx...,NaN,First address of the contiguous IP range in IP...,2,NaN
1,Ending IP,Mandatory Entry\n\nIPv4_Address Validation\nxx...,NaN,Last address of the contiguous IP range that c...,2,NaN
2,Name/Purpose,Optional Entry\n\nValidation\nSubstitute with ...,NaN,A short description of the address block that ...,2,NaN
3,Visible to EINSTEIN?,Mandatory Entry\n\nValidation\nMust be one of ...,Yes\nNo\nUnknown,Is this address block presented to EINSTEIN fo...,2,NaN
4,External Organization,Optional Entry\n\nValidation\nSubstitute with ...,NaN,If this address range is associated with an ex...,2,NaN
0,Network Aggregation TAP Identifier,Mandatory Entry\n\nValidation\nThe field is re...,NaN,Your organization’s identifier for the EINSTEI...,3,NaN
1,Site Identifier,Mandatory Entry\n\nValidation\nThe field is re...,NaN,The identifier of the data center or facility ...,3,NaN
2,Logical Location,Mandatory Entry\n\nValidation\nOne of the enum...,Outside Perimeter Router\nOutside Perimeter Fi...,Select the zone from the figure. If the locati...,3,NaN
3,Other Logical Location,Optional Entry\n\nValidation\n- If the “Logica...,NaN,"If you selected “Other” for Logical Location, ...",3,NaN
4,EINSTEIN System,Mandatory Entry\n\nValidation\nOne of the enum...,E1\nE2\nE1 & E2,Indicate what type of EINSTEIN device this TAP...,3,NaN


In [39]:
dirr= r'D:\dev\CyberScope\CyberScopeBranch\CSwebdev\code\CyberScope\EINSTEIN\2021' 
templates = []
ffilter = '2021_A_EINSTEIN_0' 
files = [f for f in os.scandir(dirr) if ffilter in f.path ]    
for file in files:  
    with open(file.path, encoding="utf-8", errors='ignore', mode='r' ) as f: 
        templates.append({'d':f'{dirr}\\{file.name}', 'c':f.read() })   

In [57]:
for i, r in df[1:2].iterrows(): 
    for t in templates:   
        dd = t['d'].replace('_0',f'_{i+0}') 
        dd = dd.replace('NCEinsteinPublicIP',f"{r['QuestionTypeCode']}" )
        cc = t['c'].replace('_0',f'_{i+0}') 
        cc = cc.replace('NCEinsteinPublicIP',f'EinsteinPublicIP') 
        cc = cc.replace('EinsteinPublicIP',r['QuestionTypeCode']) 
        cc = cc.replace('2110',str(r['QGROUP'])) 
        cc = cc.replace('20850',str(r['QPK'])) 
        print( dd )
        print( cc )
        with open(dd, 'w', encoding="utf-8") as writer:
            writer.write(cc)
 

D:\dev\CyberScope\CyberScopeBranch\CSwebdev\code\CyberScope\EINSTEIN\2021\2021_A_EINSTEIN_1.aspx
﻿<%@ Page Title="" Language="vb" AutoEventWireup="false" MasterPageFile="~/Form.Master" 
    CodeBehind="2021_A_EINSTEIN_1.aspx.vb" Inherits="CyberScope._2021_A_EINSTEIN_1" %>

<%@ Register Src="../../CustomControls/CBlabel.ascx" TagName="CBlabel" TagPrefix="uc1" %>
<%@ Register Src="../../CustomControls/CBoptionalLinks.ascx" TagName="CBoptionalLinks" TagPrefix="uc2" %>
<%@ Register Src="../../CustomControls/NCEinsteinUnannounced.ascx" TagName="NCEinsteinUnannounced" TagPrefix="uc3" %>
 
<asp:Content ID="Content1" ContentPlaceHolderID="ContentPlaceHolder1" runat="Server"> 
    <asp:Panel ID="Panel1" runat="server"> 
        <table class="table table-sm"> 
			<tr>
                <td colspan="6" class="SectionHead">
                    <uc1:CBlabel ID="CBlabelGroup" runat="server" LabelType="Group" PK_key="2111" />
                </td>
            </tr> 
			<tr>
                <td style="w

In [59]:
dirr= r"D:\dev\CyberScope\CyberScopeBranch\CSwebdev\code\CyberScope\CustomControls"
templates = []
ffilter = 'NCEinsteinPublicIP' 
files = [f for f in os.scandir(dirr) if ffilter in f.path ]    
for file in files:  
    with open(file.path, encoding="utf-8", errors='ignore', mode='r' ) as f: 
        templates.append({'d':f'{dirr}\\{file.name}', 'c':f.read() })   

In [61]:
# templates[0]

In [62]:
for i, r in df[1:2].iterrows(): 
    for t in templates: 
        dd = t['d']
        cc = t['c']
        dd = dd.replace('NCEinsteinPublicIP',f"{r['QuestionTypeCode']}" )  
        cc = cc.replace('NCEinsteinPublicIP',f"{r['QuestionTypeCode']}")  
        cc = cc.replace('EinsteinPublicIP',f"{r['QTTable']}") 
        print( dd )
        print( cc )
        with open(dd, 'w', encoding="utf-8") as writer:
            writer.write(cc)

D:\dev\CyberScope\CyberScopeBranch\CSwebdev\code\CyberScope\CustomControls\NCEinsteinUnannounced.ascx
﻿<%@ Control Language="vb" AutoEventWireup="false" CodeBehind="NCEinsteinUnannounced.ascx.vb" Inherits="CyberScope.NCEinsteinUnannounced" %>
<%@ Register TagPrefix="CB" Namespace="CyberBalance.CS.Web.UI" Assembly= "CyberBalance.CS.Web.UI" %>


<asp:BulletedList ID="bl_Errors" runat="server" Visible="false" Width="100%" 
    ForeColor="Red" BulletStyle="NotSet" CssClass="leftalign">
</asp:BulletedList>

<%-- CONFIG --%>
<CB:EinsteinDataImporter 
    ID="EinsteinDataImporter" 
    TableName="EinsteinUnannounced"  
    SprocName="EinsteinUnannounced_CRUD"  
    OnOnRowValidating="DataImporter_RowValidating" UseUploader="True"  runat="server">  
    <DataFields>
        <CB:DataField  DBColumnName="StartingIP"  runat="server"/>  
        <CB:DataField  DBColumnName="EndingIP"   runat="server"/>  
        <CB:DataField  DBColumnName="TIC_MTIPS"  PickListTypeID="332"  runat="server"/> 
     